# Task 2: Understand body language by gesture recognition with convolutional neural network

## 1. Do literature search on Convolution Neural Network. Learn how to build a convolutional layer in PyTorch.

## 2. Referring to the guide in Task 1, build your own network for gesture classification using convolutional layers. Please see the references 4 in the manual to learn how to build convolutional layers in PyTorch.

## 3. Analyse and comment on the performance of the model. Make a comparison between the fully connected based and convolutional based models and comment on it.

---------------------------------------------------------------------------------------------------------------

##  1.Importing the necessary packages

In [1]:
import numpy as np
import math
import pandas as pd

# for creating train and test set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
import torch.nn as nn
import torch.utils.data as utils_data
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torchvision import transforms
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [2]:
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import glob
import os
import itertools
import cv2

## 2.Data Processing

Define a function to preprocess the images including resizing and binaryzation

In [4]:
def processSkinImage(filePath, resize_HW=48):
    # step 1
    # read the image
    original = cv2.imread(filename=filePath)

    # step 2
    # resize the image to
    image_resized = cv2.resize(original, (resize_HW, resize_HW))

    # step 3
    # convert the image from rgb to YCbCr
    image_ycbcr = cv2.cvtColor(image_resized, cv2.COLOR_BGR2YCR_CB)

    # step 4
    # get the central color of the image
    # expected the hand to be in the central of the image
    Cb_center_color = image_ycbcr[int(resize_HW/2), int(resize_HW/2), 1]
    Cr_center_color = image_ycbcr[int(resize_HW/2), int(resize_HW/2), 2]
    # set the range
    Cb_Difference = 15
    Cr_Difference = 10

    # step 5
    # detect skin pixels
    Cb = image_ycbcr[:, :, 1]
    Cr = image_ycbcr[:, :, 2]
    index = np.where((Cb >= Cb_center_color-Cb_Difference) & (Cb <= Cb_center_color+Cb_Difference)
                     & (Cr >= Cr_center_color-Cr_Difference) & (Cr <= Cr_center_color+Cr_Difference))

    # Mark detected pixels and output
    image_output = np.zeros((resize_HW, resize_HW))
    image_output[index] = 255

    # show image
    # cv2.imshow("", image_output)
    # cv2.waitKey(0)
    return image_output

Deal with all the images using the function defined above. The processed data is stored in a new folder 'dataset_processed'.

Generate labels for each class. (class 0, 1, ..., num_classes)

In [5]:
path = './dataset/images'
path_processed = './dataset_processed/images'

# -------------------images processing--------------
for mainDir, subDir, fileList in os.walk(path):
    for file in fileList:
        currentPath = os.path.join(mainDir, file)
        processedImage = processSkinImage(currentPath)

        new_mainDir = path_processed + mainDir.split(path)[-1]
        if not os.path.exists(new_mainDir):
            os.makedirs(new_mainDir)
        cv2.imwrite(os.path.join(new_mainDir, file), processedImage)

# -----------------label generation----------------
label_path = './dataset_processed/labels'
if not os.path.exists(label_path):
    os.makedirs(label_path)

files = os.listdir(path)
for i, file in enumerate(files):
    subclass_label_path = os.path.join(label_path, file+'.txt')
    with open(subclass_label_path, 'w') as f:
        f.write('#label\n')
    for _ in range(len(os.listdir(os.path.join(path_processed, file)))):
        with open(subclass_label_path, 'a') as f:
            f.write('{:d}\n'.format(i))
    f.close()

## 3. Load Data

In [6]:
Image = [] #x
path_images = './dataset_processed/images'
for mainDir, subDir, fileList in os.walk(path_images):
    for file in fileList:
        currentPath = os.path.join(mainDir, file)
        Image.append(cv2.imread(currentPath)[:, :, 0])
Image = np.array(Image)
dataset_size, H, W = Image.shape
# for FCNN model, the image need to be stretched into one dimension: (b, h, w)->(b, h*w)
Image = Image.reshape(dataset_size, 1, H, W)


Label = []  #y
path_labels = './dataset_processed/labels'
for file in os.listdir(path_labels):
    Label.append(np.loadtxt(os.path.join(path_labels, file)))
Label = np.array(list(itertools.chain.from_iterable(Label)))
num_classes = int(np.max(Label))+1

In [7]:
print("Images loaded: ", len(Image))
print("Labels loaded: ", len(Label))

Images loaded:  78
Labels loaded:  78


##  3. Create train and test set

In [8]:
# Hyper parameters
num_classes = 4
batch_size = 25
learning_rate = 0.01

In [9]:
dataset = utils_data.TensorDataset(torch.Tensor(Image), torch.LongTensor(Label))
split_ratio = 0.8
train_size = int(split_ratio * dataset_size)
test_size = dataset_size - train_size
train_set, test_set = utils_data.random_split(dataset, [train_size, test_size])
train_loader = utils_data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = utils_data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)
print('Data is ready!')

Data is ready!


##  4. Implementing CNNs

In [10]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1, padding=0) #1,4
        self.relu1 = nn.ReLU()        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=5, stride=1, padding=0) #4,8
        self.relu2 = nn.ReLU()      
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(648, 25) #8,9,9
        self.fc2 = nn.Linear(25, 4) #8,9,9
        
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Max pool 2 
        out = self.maxpool2(out)     
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        x= self.fc2(out)
        return x

    

## 5. Loss and Optimization

In [11]:
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
loss_func = nn.CrossEntropyLoss()

## 6. Train and Test the data using CNN model

In [12]:
best_accuracy = 0
for epoch in range(51):
    running_loss = 0.0
    train_acc = 0.0
    for step, (batch_image, batch_label) in enumerate(train_loader):
        model.train()
        # if torch.cuda.is_available():
        #     batch_image, batch_label = batch_image.cuda(), batch_label.cuda()
        batch_output = model(batch_image)
        batch_loss = loss_func(batch_output, batch_label)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        running_loss += batch_loss.item()

        # train accuracy
        _, train_predicted = torch.max(batch_output.data, 1)
        train_acc += (train_predicted == batch_label).sum().item()

    train_acc /= train_size
    running_loss /= (step+1)

    # ----------test----------
    model.eval()
    test_acc = 0.0
    for test_image, test_label in test_loader:
        test_output = model(test_image)
        _, predicted = torch.max(test_output.data, 1)
        test_acc += (predicted == test_label).sum().item()
    test_acc /= test_size

    print('epoch={:d}\ttrain loss={:.6f}\ttrain accuracy={:.3f}\ttest accuracy={:.3f}'.format(
        epoch, running_loss, train_acc, test_acc))
    if test_acc >= best_accuracy:
        torch.save(model.state_dict(), 'FCNN_model.pkl')
        best_accuracy = test_acc

epoch=0	train loss=65.900239	train accuracy=0.387	test accuracy=0.250
epoch=1	train loss=2.269722	train accuracy=0.484	test accuracy=0.688
epoch=2	train loss=0.612733	train accuracy=0.726	test accuracy=0.938
epoch=3	train loss=0.169063	train accuracy=0.968	test accuracy=0.938
epoch=4	train loss=0.083873	train accuracy=0.984	test accuracy=1.000
epoch=5	train loss=0.022070	train accuracy=1.000	test accuracy=0.938
epoch=6	train loss=0.095359	train accuracy=0.984	test accuracy=0.938
epoch=7	train loss=0.009064	train accuracy=1.000	test accuracy=1.000
epoch=8	train loss=0.018255	train accuracy=0.984	test accuracy=0.938
epoch=9	train loss=0.004835	train accuracy=1.000	test accuracy=0.938
epoch=10	train loss=0.002859	train accuracy=1.000	test accuracy=0.938
epoch=11	train loss=0.002915	train accuracy=1.000	test accuracy=0.938
epoch=12	train loss=0.001990	train accuracy=1.000	test accuracy=0.938
epoch=13	train loss=0.001572	train accuracy=1.000	test accuracy=0.938
epoch=14	train loss=0.000933	